In [1]:
import os
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras import backend

def get_pcam_generators(base_dir, train_batch_size=32, val_batch_size=32):

     # dataset parameters
     train_path = os.path.join(base_dir, 'train+val','train')
     valid_path = os.path.join(base_dir, 'train+val','valid')

     # instantiate data generators
     datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

     train_gen = datagen.flow_from_directory(train_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=train_batch_size,
                                             class_mode='binary')

     val_gen = datagen.flow_from_directory(valid_path,
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=val_batch_size,
                                             class_mode='binary')

     return train_gen, val_gen

# the size of the images in the PCAM dataset
IMAGE_SIZE = 96
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)
train_gen, val_gen = get_pcam_generators('C:\\Users\s167917\\Documents\\#School\\Jaar 3\\3 Project Imaging\\data')
                                         
pretrained = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
group_nr = 0
layer_names = []
layer_group_nr = []
for i,layer in enumerate(pretrained.layers):
    layer_names.append(layer.name)
    if layer.name.find("block_"+str(group_nr+1)) !=-1:
        group_nr += 1
    layer_group_nr.append(group_nr)

#for i in range(len(pretrained.layers)):
#        print(layer_group_nr[i],layer_names[i])
    
for i in range(19):
    model_name = 'MNV2_blocks_experiment_{:02d}'.format(i)
    print("\n"+model_name)
    input = Input(input_shape)
    if i != 18:
        # imagenet weights and freezing of blocks
        pretrained = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
        for j in range(len(pretrained.layers)):
            if layer_group_nr[j] >= i:
                pretrained.layers[j].trainable = True
            else:
                pretrained.layers[j].trainable = False                
    else:
        # random initialization of weights
        pretrained = MobileNetV2(input_shape=input_shape, include_top=False, weights=None)
        
    # Check the trainable status of the individual layers
    for layer in pretrained.layers:
        print(layer.trainable,layer.name,layer)
    
    output = pretrained(input)
    output = GlobalAveragePooling2D()(output)
    output = Dropout(0.5)(output)
    output = Dense(1, activation='sigmoid')(output)

    model = Model(input, output)
    model.compile(SGD(lr=0.001, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])
    
    # print a summary of the model on screen
    model.summary()
    
    # save the model and weights    
    model_filepath = model_name + '.json'
    weights_filepath = model_name + '_weights.hdf5'
    
    model_json = model.to_json() # serialize model to JSON
    with open(model_filepath, 'w') as json_file:
        json_file.write(model_json)
        
    # define the model checkpoint and Tensorboard callbacks
    checkpoint = ModelCheckpoint(weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    tensorboard = TensorBoard(os.path.join('logs', model_name))
    callbacks_list = [checkpoint, tensorboard]
    
    # train the model, note that we define "mini-epochs"
    train_steps = train_gen.n//train_gen.batch_size//20
    val_steps = val_gen.n//val_gen.batch_size//20
    
    # since the model is trained for only 10 "mini-epochs", i.e. half of the data is
    # not used during training
    history = model.fit_generator(train_gen, steps_per_epoch=train_steps,
                        validation_data=val_gen,
                        validation_steps=val_steps,
                        epochs=10,
                        callbacks=callbacks_list)
    backend.clear_session()

C:\Users\s167917\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 144000 images belonging to 2 classes.
Found 16000 images belonging to 2 classes.

MNV2_blocks_experiment_00
True input_3 <keras.engine.input_layer.InputLayer object at 0x000001FF766E16D8>
True Conv1_pad <keras.layers.convolutional.ZeroPadding2D object at 0x000001FF7822BB38>
True Conv1 <keras.layers.convolutional.Conv2D object at 0x000001FF78217278>
True bn_Conv1 <keras.layers.normalization.BatchNormalization object at 0x000001FF781DA278>
True Conv1_relu <keras.layers.advanced_activations.ReLU object at 0x000001FF780FD9B0>
True expanded_conv_depthwise <keras.layers.convolutional.DepthwiseConv2D object at 0x000001FF7818B9B0>
True expanded_conv_depthwise_BN <keras.layers.normalization.BatchNormalization object at 0x000001FF7822B2E8>
True expanded_conv_depthwise_relu <keras.layers.advanced_activations.ReLU object at 0x000001FF7683EEB8>
True expanded_conv_project <keras.layers.convolutional.Conv2D object at 0x000001FF7685DB70>
True expanded_conv_project_BN <keras.layers.normalization.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
mobilenetv2_1.00_96 (Model)  (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 2,225,153
Non-trainable params: 34,112
_________________________________________________________________
Epoch 1/10
225/225 [==============================] - 179s 797ms/step - loss: 0.4727 - acc: 0.7942 - val_loss: 0.4141 - val_acc

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
mobilenetv2_1.00_96 (Model)  (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 2,223,329
Non-trainable params: 35,936
_________________________________________________________________
Epoch 1/10
225/225 [==============================] - 168s 745ms/step - loss: 0.4644 - acc: 0.8047 - val_loss: 0.6071 - val_acc

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
mobilenetv2_1.00_96 (Model)  (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 2,218,193
Non-trainable params: 41,072
_________________________________________________________________
Epoch 1/10
225/225 [==============================] - 158s 702ms/step - loss: 0.4629 - acc: 0.8008 - val_loss: 1.4519 - val_acc

...

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
mobilenetv2_1.00_96 (Model)  (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,527,361
Non-trainable params: 731,904
_________________________________________________________________
Epoch 1/10
225/225 [==============================] - 158s 702ms/step - loss: 0.4928 - acc: 0.7828 - val_loss: 0.6743 - val_ac

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
mobilenetv2_1.00_96 (Model)  (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,207,361
Non-trainable params: 1,051,904
_________________________________________________________________
Epoch 1/10
225/225 [==============================] - 165s 735ms/step - loss: 0.4966 - acc: 0.7799 - val_loss: 0.5497 - val_

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
mobilenetv2_1.00_96 (Model)  (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 887,361
Non-trainable params: 1,371,904
_________________________________________________________________
Epoch 1/10
225/225 [==============================] - 157s 696ms/step - loss: 0.4827 - acc: 0.7826 - val_loss: 0.6885 - val_ac

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
mobilenetv2_1.00_96 (Model)  (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________
Epoch 1/10
225/225 [==============================] - 163s 726ms/step - loss: 0.5559 - acc: 0.7399 - val_loss: 0.5336 - val_acc:

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 96, 3)         0         
_________________________________________________________________
mobilenetv2_1.00_96 (Model)  (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 2,225,153
Non-trainable params: 34,112
_________________________________________________________________
Epoch 1/10
225/225 [==============================] - 163s 723ms/step - loss: 0.6032 - acc: 0.6896 - val_loss: 4.1270 - val_acc